### Test M3 with simulations
#### E Flynn
#### 6/10/2019

I updated the model to allow for effects that are sex-differential (rather than sex-specific).

In [1]:
source("project_utils.R")
library(rstan)
set.seed(610)

Loading required package: MASS
Loading required package: Matrix
Loading required package: mnormt
Loading required package: ggplot2
Loading required package: StanHeaders
rstan (Version 2.18.2, GitRev: 2e1f913d3ca3)
For execution on a local, multicore CPU with excess RAM we recommend calling
options(mc.cores = parallel::detectCores()).
To avoid recompilation of unchanged Stan programs, we recommend calling
rstan_options(auto_write = TRUE)


Below is a function that simulates a five-component model - with fourth component w var-covar matrix, and fifth component containing half scaled 1.1 for f and half scaled 1.1 for m.

In [2]:
model3sim <- function(N, p, sigmasq, S, s, R){
    # sigmasq is a vector of three variances
    # S is a variance-covariance matrix
    # s is the scaling parameter
    
    Sigma <- nearPD(S)$mat # nearest positive definite matrix
    zeros <- c(0,0)
    
    # sample squared SEs
    se2 <- simSE2(N)

    ### SAMPLE BETAS
    # M0
    n.m0 <- round(p[1]*N)
    se.m0 <- matrix(se2[1:(2*n.m0)], n.m0, 2)
    betas.m0 <- do.call(rbind, lapply(1:n.m0, function(x) mvrnorm(1, zeros, diag(se.m0[x,]))))

    # M1 
    n.m1 <- round(p[2]*N)
    se.m1 <- matrix(se2[(2*n.m0+1):(2*(n.m0+n.m1))], n.m1, 2)
    betas.m1 <- do.call(rbind, lapply(1:n.m1, function(x) 
        mvrnorm(1, zeros, diag(se.m1[x,])+diag(c(sigmasq[1], 0)))))

    # M2
    n.m2 <- round(p[3]*N)    
    se.m2 <- matrix(se2[(2*(n.m0+n.m1)+1):(2*(n.m0+n.m1+n.m2))], n.m2, 2)
    betas.m2 <- do.call(rbind, lapply(1:n.m2, function(x) 
        mvrnorm(1, zeros, diag(se.m2[x,])+diag(c(0,sigmasq[2])))))

    # M3
    n.m3 <- round(p[4]*N)   
    se.m3 <- matrix(se2[(2*(n.m0+n.m1+n.m2)+1):(2*(n.m0+n.m1 + n.m2 + n.m3))], n.m3, 2)
    betas.m3 <- do.call(rbind, lapply(1:n.m3, function(x) 
        mvrnorm(1, zeros, diag(se.m3[x,])+Sigma))) # variance-cov matrix

    #  scaling component - do half from f scaled + half from m scaled
    n.m4.1 <- round(p[5]*N*0.5)
    prev.count <- n.m0+n.m1 + n.m2 + n.m3 + n.m4.1
    n.m4.2 <-  N-prev.count
    se.m4.1 <- matrix(se2[(2*(n.m0+n.m1 + n.m2 + n.m3)+1):(2*prev.count)], n.m4.1, 2)

    se.m4.2 <- matrix(se2[(2*(prev.count)+1):(2*N)], n.m4.2, 2)
    betas.m4.1 <- do.call(rbind, lapply(1:n.m4.1, function(x) 
        mvrnorm(1, zeros, diag(se.m4.1[x,])+matrix(c((s**2)*sigmasq[3], s*1*sigmasq[3],s*1*sigmasq[3],sigmasq[3]),2,2)))) # TODO - make sure correct

    # half up in female
    betas.m4.2 <- do.call(rbind, lapply(1:(n.m4.2), function(x) 
        mvrnorm(1, zeros, diag(se.m4.2[x,])+ matrix(c(sigmasq[3], s*1*sigmasq[3], s*1*sigmasq[3],(s**2)*sigmasq[3]), 2, 2))))
    

    # put together
    betas <- do.call(rbind, list(betas.m0, betas.m1, betas.m2, betas.m3, betas.m4.1, betas.m4.2))
    ses <- do.call(rbind, list(se.m0, se.m1, se.m2, se.m3, se.m4.1, se.m4.2))
        
    cov.data.k4.sim <- list(
        N = N,
        M = 2,
        B = betas,
        SE = ses,
        K = 6
        # change tthis back to 5 
    )
    return(cov.data.k4.sim)
}                                                 


In [3]:
N <- 2000
#p <- c(0.5, 0.05, 0.1, 0.15, 0.1)
p <- c(0.2, 0.2, 0.1, 0.05, 0.45 ) 
sigmasq <- c(0.02, 0.01, 0.03)
S <- matrix(c(0.012,0.01095445, 0.01095445, 0.010),2,2)
R <- matrix(c(1,.999,.999,1), 2, 2)
cov.data.M3.sim <- model3sim(N, p, sigmasq, S, 1.1, R)

In [4]:
cov.data.M3.sim

-0.0042955935,0.0097544546
0.0227049165,0.0848137999
0.0059035019,-0.0056791672
0.0159848945,-0.0039910759
-0.0094764683,-0.0354088082
-0.0149405674,0.0057968225
0.0027123788,-0.0406712920
-0.0190923913,0.0009336549
0.0639494375,0.0046954255
-0.0218055509,-0.0036254835
0.0050648324,0.0003276355


In [5]:
options(warn=-1)
fit3_sim <- stan(
  file =   '../models/model4_v1.stan', # Stan program - TODO: fill in - don't have a model working for this yet
  data = cov.data.M3.sim,    # named list of data
  chains = 4,             # number of Markov chains
  warmup = 200,          # number of warmup iterations per chain
  iter = 400,            # total number of iterations per chain             
  refresh = 100          # show progress every 'refresh' iterations
  )


SAMPLING FOR MODEL 'model4_v1' NOW (CHAIN 1).
Chain 1: 
Chain 1: Gradient evaluation took 0.04 seconds
Chain 1: 1000 transitions using 10 leapfrog steps per transition would take 400 seconds.
Chain 1: Adjust your expectations accordingly!
Chain 1: 
Chain 1: 
Chain 1: Iteration:   1 / 400 [  0%]  (Warmup)
Chain 1: Iteration: 100 / 400 [ 25%]  (Warmup)
Chain 1: Iteration: 200 / 400 [ 50%]  (Warmup)
Chain 1: Iteration: 201 / 400 [ 50%]  (Sampling)
Chain 1: Iteration: 300 / 400 [ 75%]  (Sampling)
Chain 1: Iteration: 400 / 400 [100%]  (Sampling)
Chain 1: 
Chain 1:  Elapsed Time: 449.33 seconds (Warm-up)
Chain 1:                133.85 seconds (Sampling)
Chain 1:                583.18 seconds (Total)
Chain 1: 

SAMPLING FOR MODEL 'model4_v1' NOW (CHAIN 2).
Chain 2: 
Chain 2: Gradient evaluation took 0.03 seconds
Chain 2: 1000 transitions using 10 leapfrog steps per transition would take 300 seconds.
Chain 2: Adjust your expectations accordingly!
Chain 2: 
Chain 2: 
Chain 2: Iteration:   1 / 

In [6]:
print(fit3_sim, pars=c("sigmass","sigmaf","sigmam","Omegacor","L_Omega","sigmasq", "Sigma", "pi", "lp__"), probs=c(0.1, 0.025, 0.975), digits_summary = 5)

Inference for Stan model: model4_v1.
4 chains, each with iter=400; warmup=200; thin=1; 
post-warmup draws per chain=200, total post-warmup draws=800.

                    mean se_mean      sd        10%       2.5%      97.5% n_eff
sigmass[1]       0.19122 0.03265 0.49226    0.06594    0.03660    0.75293   227
sigmass[2]       0.19122 0.03265 0.49226    0.06594    0.03660    0.75293   227
sigmaf[1]        0.17188 0.00060 0.01169    0.15841    0.15198    0.19893   385
sigmaf[2]        0.18625 0.00073 0.01348    0.17051    0.16520    0.21889   340
sigmam[1]        0.18559 0.00059 0.01144    0.17203    0.16643    0.21196   376
sigmam[2]        0.20237 0.00069 0.01276    0.18685    0.18123    0.23223   343
Omegacor[1,1]    1.00000     NaN 0.00000    1.00000    1.00000    1.00000   NaN
Omegacor[1,2]    0.99922 0.00002 0.00044    0.99864    0.99812    0.99985   708
Omegacor[2,1]    0.99922 0.00002 0.00044    0.99864    0.99812    0.99985   708
Omegacor[2,2]    1.00000     NaN 0.00000    1.000

In [ ]:
p <- c(0.2, 0.2, 0.1, 0.05, 0.45 ) 

OTHER MODELS
- model2 <- model2Sim
- model2 with variance covariance matrix <- model2Simv2
- mvpmm model
- mvpmm + k4 mat (3 component)
- mvpmm + var-cov mat (3 component)
- m2 with mvpmm as k4 (4 component)
- m2 with var-cov + mvpmm sigma mat (5 component) <- above
- m2 with k4 + mvpmm sigma mat (5 component)


In [ ]:
m2.sim <- model2sim(N, p, sigmasq) # where sigmasq is a vector of FOUR variances - function is in project_utils.R
# the fourth component has zero off-diagonals

In [ ]:
model2Simv2 <- function(N, p, sigmasq, S){ # fourth component HAS offdiagonals, this is from S matrix
    # sigmasq is a vector of two variances
    # S is a variance-covariance matrix
    
    Sigma <- nearPD(S)$mat # nearest positive definite matrix
    zeros <- c(0,0)
    
    # sample squared SEs
    se2 <- simSE2(N)

    ### SAMPLE BETAS
    # M0
    n.m0 <- round(p[1]*N)
    se.m0 <- matrix(se2[1:(2*n.m0)], n.m0, 2)
    betas.m0 <- do.call(rbind, lapply(1:n.m0, function(x) mvrnorm(1, zeros, diag(se.m0[x,]))))

    # M1 
    n.m1 <- round(p[2]*N)
    se.m1 <- matrix(se2[(2*n.m0+1):(2*(n.m0+n.m1))], n.m1, 2)
    betas.m1 <- do.call(rbind, lapply(1:n.m1, function(x) 
        mvrnorm(1, zeros, diag(se.m1[x,])+diag(c(sigmasq[1], 0)))))

    # M2
    n.m2 <- round(p[3]*N)    
    se.m2 <- matrix(se2[(2*(n.m0+n.m1)+1):(2*(n.m0+n.m1+n.m2))], n.m2, 2)
    betas.m2 <- do.call(rbind, lapply(1:n.m2, function(x) 
        mvrnorm(1, zeros, diag(se.m2[x,])+diag(c(0,sigmasq[2])))))

    # M3
    n.m3 <- N-(n.m0+n.m1 + n.m2)
    se.m3 <- matrix(se2[(2*(n.m0+n.m1+n.m2)+1):(2*N)], n.m3, 2)
    betas.m3 <- do.call(rbind, lapply(1:n.m3, function(x) 
        mvrnorm(1, zeros, diag(se.m3[x,])+Sigma)))


    # put together
    betas <- do.call(rbind, list(betas.m0, betas.m1, betas.m2, betas.m3))
    ses <- do.call(rbind, list(se.m0, se.m1, se.m2, se.m3))
        
    cov.data.k4.sim <- list(
        N = N,
        M = 2,
        B = betas,
        SE = ses,
        K = 4
    )
    return(cov.data.k4.sim)
}               